# [Manifold tutorial](https://nbviewer.org/github/sagemanifolds/SageManifolds/blob/master/Notebooks/SM_tutorial.ipynb) from SageManifolds Tutorial

In [2]:
version()
print(SAGE_ROOT)

/home/topolo/TQFT/sage


Set up the notebook to display mathematical objects using LaTeX rendering.

In [3]:
%display latex

## Defining a manifold
Define a differentiable manifold of dimension 3 over $\mathbb{R}$.

In [4]:
M = Manifold(3, 'M', latex_name=r'\mathcal{M}', start_index=1)

In [5]:
print(M)
print(type(M))
M

3-dimensional differentiable manifold M
<class 'sage.manifolds.differentiable.manifold.DifferentiableManifold_with_category'>


3-dimensional differentiable manifold M

We can ask for the category of M and see that it's the category of smooth manifolds over $\mathbb{R}$.

In [6]:
category(M)

Category of smooth manifolds over Real Field with 53 bits of precision